In [ ]:
!pip install sentence-transformers==2.7.0 datasets==2.19.0 huggingface_hub==0.23.0 faiss-cpu==1.8.0 -qqq

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.0/44.0 kB 3.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 3.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 3.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 3.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.1/44.1 kB 3.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.5/43.5 kB 2.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.1/44.1 kB 3.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.1/44.1 kB 3.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.1/44.1 kB 3.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 3.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 3.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 3.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

# **사전 학습된 언어 모델을 불러와 문장 임베딩 모델 만들기**

In [ ]:
from sentence_transformers import SentenceTransformer, models

# 허깅페이스 임베딩 모델 로드
transformer_model = models.Transformer('klue/roberta-base')

# 풀링층 생성
pooling_layer = models.Pooling(
    transformer_model.get_word_embedding_dimension(),
    # 평균 풀링 모드
    pooling_mode_mean_tokens=True
)

embedding_model = SentenceTransformer(modules=[transformer_model, pooling_layer])

/usr/local/lib/python3.11/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/546 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/443M [00:00<?, ?B/s]

Some weights of RobertaModel were not initialized from the model checkpoint at klue/roberta-base and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


tokenizer_config.json:   0%|          | 0.00/375 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/248k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/752k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/173 [00:00<?, ?B/s]

# **실습 데이터셋 다운로드 및 확인**

In [ ]:
from datasets import load_dataset

klue_sts_train = load_dataset('klue', 'sts', split='train')
klue_sts_test = load_dataset('klue', 'sts', split='validation')
klue_sts_train[0]

# {'guid': 'klue-sts-v1_train_00000',
#  'source': 'airbnb-rtt',
#  'sentence1': '숙소 위치는 찾기 쉽고 일반적인 한국의 반지하 숙소입니다.',
#  'sentence2': '숙박시설의 위치는 쉽게 찾을 수 있고 한국의 대표적인 반지하 숙박시설입니다.',
#  'labels': {'label': 3.7, 'real-label': 3.714285714285714, 'binary-label': 1}}

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/11668 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/519 [00:00<?, ? examples/s]

{'guid': 'klue-sts-v1_train_00000',
 'source': 'airbnb-rtt',
 'sentence1': '숙소 위치는 찾기 쉽고 일반적인 한국의 반지하 숙소입니다.',
 'sentence2': '숙박시설의 위치는 쉽게 찾을 수 있고 한국의 대표적인 반지하 숙박시설입니다.',
 'labels': {'label': 3.7, 'real-label': 3.714285714285714, 'binary-label': 1}}

# **학습 데이터에서 검증 데이터셋 분리하기**

In [ ]:
# 학습 데이터셋의 10%를 검증 데이터셋으로 구성한다.
klue_sts_train = klue_sts_train.train_test_split(test_size=0.1, seed=42)
klue_sts_train, klue_sts_eval = klue_sts_train['train'], klue_sts_train['test']

# **label 정규화하기**

In [ ]:
from sentence_transformers import InputExample

# 유사도 점수를 0~1 사이로 정규화 하고 InputExample 객체에 담는다.
# STS 학습 방법을 위해 InputExample 객체 사용 필요
def prepare_sts_examples(dataset):
  examples = []

  for data in dataset:
    examples.append(
        InputExample(
            texts=[data['sentence1'], data['sentence2']],
            # 데이터셋의 유사도 점수는 0 ~ 5 범위를 가짐
            # Sentence-BERT 모델에서는 유사도 점수를 0 ~ 1 사이의 값으로 사용해야 함
            # 따라서 5로 나눠
            label=data['labels']['label'] / 5.0
        )
    )

  return examples

train_examples = prepare_sts_examples(klue_sts_train)
eval_examples = prepare_sts_examples(klue_sts_eval)
test_examples = prepare_sts_examples(klue_sts_test)

# **학습에 사용할 배치 데이터셋 만들기**

In [ ]:
from torch.utils.data import DataLoader

train_dataloader = DataLoader(train_examples, shuffle=True, batch_size=16)

# **검증을 위한 평가 객체 준비**

In [ ]:
from sentence_transformers.evaluation import EmbeddingSimilarityEvaluator

# from_input_examples -> InputExample 리스트를 받아 평가자(Evaluator) 객체를 생성
# 매개변수 데이터로 평가를 할 수 있는 객체 리턴
eval_evaluator = EmbeddingSimilarityEvaluator.from_input_examples(eval_examples)
test_evaluator = EmbeddingSimilarityEvaluator.from_input_examples(test_examples)

# **언어 모델을 그대로 활용할 경우 문장 임베딩 모델의 성능**

In [ ]:
test_evaluator(embedding_model)
# 0.36460670798564826

0.36460670798564826

# **임베딩 모델 학습**

In [ ]:
# 손실 함수
from sentence_transformers import losses

num_epochs = 4
model_name = 'klue/roberta-base'
model_save_path = 'output/training_sts_' + model_name.replace("/", "-")
# CosineSimilarityLoss -> 문장 임베딩의 코사인 유사도를 학습하도록 유도하는 손실 함수
train_loss = losses.CosineSimilarityLoss(model=embedding_model)

# 임베딩 모델 학습
embedding_model.fit(
    # 학습 데이터와 손실 함수 설정
    train_objectives=[(train_dataloader, train_loss)],
    # 평가 도구 설정
    evaluator=eval_evaluator,
    # 에포크 횟수(반복 횟수)
    epochs=num_epochs,
    # 1000 스텝마다 평가 진행
    evaluation_steps=1000,
    # 초반 100 스텝 동안 학습률 점진적 증가 (학습 안정화)
    warmup_steps=100,
    # 저장 경로
    output_path=model_save_path
)

Epoch:   0%|          | 0/4 [00:00<?, ?it/s]

Iteration:   0%|          | 0/657 [00:00<?, ?it/s]

Iteration:   0%|          | 0/657 [00:00<?, ?it/s]

Iteration:   0%|          | 0/657 [00:00<?, ?it/s]

Iteration:   0%|          | 0/657 [00:00<?, ?it/s]

# **학습한 임베딩 모델의 성능 평가**

In [ ]:
trained_embedding_model = SentenceTransformer(model_save_path)
test_evaluator(trained_embedding_model)
# 0.8965595666246748

0.8896156291254342

# **허깅페이스 허브에 모델 저장**

In [ ]:
from huggingface_hub import login
from huggingface_hub import HfApi

# 허깅페이스 로그인
login(token='허깅페이스 허브 토큰 입력')
# HfApi -> 모델 업로드와 관련된 API 객체 생성
api = HfApi()
repo_id = "klue-roberta-base-klue-sts"
# 저장할 모델의 저장소 이름 설정
api.create_repo(repo_id=repo_id)

# upload_folder -> 허깅페이스 업로드
api.upload_folder(
    folder_path=model_save_path,
    repo_id=f"본인의 허깅페이스 아이디 입력/{repo_id}",
    # 모델 타입 -> dataset으로 하면 데이터셋 업로드
    repo_type="model",
)

UnicodeEncodeError: 'latin-1' codec can't encode characters in position 7-11: ordinal not in range(256)

# **실습 데이터를 내려받고 예시 데이터 확인**

In [ ]:
from datasets import load_dataset
klue_mrc_train = load_dataset('klue', 'mrc', split='train')
klue_mrc_train[0]
# {'title': '제주도 장마 시작 … 중부는 이달 말부터',
#  'context': '올여름 장마가 17일 제주도에서 시작됐다. 서울 등 중부지방은 예년보다 사나흘 정도 늦은 이달 말께 장마가 시작될 전망이다.17일 기상청에 따르면 제주도 남쪽 먼바다에 있는 장마전선의 영향으로 이날 제주도 산간 및 내륙지역에 호우주의보가 내려지면서 곳곳에 100㎜에 육박하는 많은 비가 내렸다. 제주의 장마는 평년보다 2~3일, 지난해보다는 하루 일찍 시작됐다. 장마는 고온다습한 북태평양 기단과 한랭 습윤한 오호츠크해 기단이 만나 형성되는 장마전선에서 내리는 비를 뜻한다.장마전선은 18일 제주도 먼 남쪽 해상으로 내려갔다가 20일께 다시 북상해 전남 남해안까지 영향을 줄 것으로 보인다. 이에 따라 20~21일 남부지방에도 예년보다 사흘 정도 장마가 일찍 찾아올 전망이다. 그러나 장마전선을 밀어올리는 북태평양 고기압 세력이 약해 서울 등 중부지방은 평년보다 사나흘가량 늦은 이달 말부터 장마가 시작될 것이라는 게 기상청의 설명이다. 장마전선은 이후 한 달가량 한반도 중남부를 오르내리며 곳곳에 비를 뿌릴 전망이다. 최근 30년간 평균치에 따르면 중부지방의 장마 시작일은 6월24~25일이었으며 장마기간은 32일, 강수일수는 17.2일이었다.기상청은 올해 장마기간의 평균 강수량이 350~400㎜로 평년과 비슷하거나 적을 것으로 내다봤다. 브라질 월드컵 한국과 러시아의 경기가 열리는 18일 오전 서울은 대체로 구름이 많이 끼지만 비는 오지 않을 것으로 예상돼 거리 응원에는 지장이 없을 전망이다.',
#  'news_category': '종합',
#  'source': 'hankyung',
#  'guid': 'klue-mrc-v1_train_12759',
#  'is_impossible': False,
#  'question_type': 1,
#  'question': '북태평양 기단과 오호츠크해 기단이 만나 국내에 머무르는 기간은?',
#  'answers': {'answer_start': [478, 478], 'text': ['한 달가량', '한 달']}}

# **기본 임베딩 모델 불러오기**

In [ ]:
from sentence_transformers import SentenceTransformer

sentence_model = SentenceTransformer('shangrilar/klue-roberta-base-klue-sts')

# **데이터 전처리**

In [ ]:
from datasets import load_dataset

klue_mrc_train = load_dataset('klue', 'mrc', split='train')
klue_mrc_test = load_dataset('klue', 'mrc', split='validation')

df_train = klue_mrc_train.to_pandas()
df_test = klue_mrc_test.to_pandas()

# 데이터 프레임에서 필요한 열만 추출
df_train = df_train[['title', 'question', 'context']]
df_test = df_test[['title', 'question', 'context']]

# **질문과 관련이 없는 기사를 irrelevant_context 컬럼에 추가**

In [ ]:
# 질문과 관련된 문맥뿐만 아니라 연관되지 않은 문맥도 함께 제공해서 모델이 피요한 정보와
불필요한 정보를 구분할 수 있도록 학습시킴
def add_ir_context(df):
  irrelevant_contexts = []

  for idx, row in df.iterrows():
    title = row['title']
    # df.query(f"title != '{title}'") -> 현재 title과 다른 title을 가진 데이터를 필터링
    # sample(n=1) -> 필터링된 데이터 중 랜덤으로 1개 선택
    # ['context'].values[0] -> 랜덤으로 선택한 행에서 context값 추출
    irrelevant_contexts.append(df.query(f"title != '{title}'").sample(n=1)['context'].values[0])

  # 해당 방식으로 입력하면 기존 데이터프레임의 각 행에 매핑되어 추가됨
  df['irrelevant_context'] = irrelevant_contexts

  return df

df_train_ir = add_ir_context(df_train)
df_test_ir = add_ir_context(df_test)

# **성능 평가에 사용할 데이터 생성**

In [ ]:
from sentence_transformers import InputExample

examples = []

for idx, row in df_test_ir[:100].iterrows():
  examples.append(
      InputExample(texts=[row['question'], row['context']], label=1)
  )

  examples.append(
      InputExample(texts=[row['question'], row['irrelevant_context']], label=0)
  )

# **기본 임베딩 모델의 성능 평가 결과**

In [ ]:
from sentence_transformers.evaluation import EmbeddingSimilarityEvaluator

# [연관, 비연관...] 데이터의 라벨을 얼마나 맞추는지 평가
evaluator = EmbeddingSimilarityEvaluator.from_input_examples(examples)
evaluator(sentence_model)

# 0.8151553052035344

# **긍정 데이터만으로 학습 데이터 구성**

In [ ]:
train_samples = []

for idx, row in df_train_ir.iterrows():
  train_samples.append(InputExample(
      texts=[row['question'], row['context']]
  ))

# **중복 학습 데이터 제거**

In [ ]:
from sentence_transformers import datasets

batch_size = 16

loader = datasets.NoDuplicatesDataLoader(train_samples, batch_size=batch_size)

# **MNR 손실 함수 불러오기**

In [ ]:
from sentence_transformers import losses

loss = losses.MultipleNegativesRankingLoss(sentence_model)

# **MRC 데이터셋으로 미세 조정**

In [ ]:
epochs = 1
save_path = './klue_mrc_mnr'

sentence_model.fit(
    train_objectives=[(loader, loss)],
    epochs=epochs,
    warmup_steps=100,
    output_path=save_path,
    show_progress_bar=True
)

# **미세 조정한 모델 성능 평가**

In [ ]:
evaluator(sentence_model)
# 0.8600968992433692

# **허깅페이스 허브에 미세 조정한 모델 업로드**

In [ ]:
from huggingface_hub import HfApi

api = HfApi()
repo_id = "klue-roberta-base-klue-sts-mrc"
api.create_repo(repo_id=repo_id)

api.upload_folder(
    folder_path=save_path,
    repo_id=f"본인의 허깅페이스 아이디 입력/{repo_id}",
    repo_type="model",
)

# **교차 인코더로 사용할 사전 학습 모델 불러오기**

In [ ]:
from sentence_transformers.cross_encoder import CrossEncoder

# SentenceTransformer로 로드 -> 바이 인코더
# CrossEncoder로 로드 -> 교차 인코더
cross_model = CrossEncoder('klue/roberta-small', num_labels=1)

# **미세 조정하지 않은 교차 인코더의 성능 평가 결과**

In [ ]:
from sentence_transformers.cross_encoder.evaluation import CECorrlationEvaluator

ce_evaluator = CRCorrelationEvaluator.from_input_examples(examples)
ce_evaluator(cross_model)
# 0.003316821814673943

# **교차 인코더 학습 데이터셋 준비**

In [ ]:
train_samples = []

for idx, row in df_train_ir.iterrows():
  train_samples.append(InputExample(
      texts=[row['question'], row['context']], lable=1
  ))

  train_samples.append(InputExample(
      texts=[row['question'], row['irrelevant_context']], label=0
  ))

# **교차 인코더 학습 수행**

In [ ]:
train_batch_size = 16
num_epochs = 1
model_save_path = 'output/training_mrc'

train_dataloader = DataLoader(train_samples, shuffle=True, batch_size=train_batch_size)

cross_model.fit(
    train_dataloade=train_dataloader,
    epochs=num_epochs,
    warmup_steps=100,
    output_path=model_save_path
)

# **학습한 교차 인코더 평가 결과**

In [ ]:
ce_evaluator(cross_model)
# 0.8650250798639563

# **학습을 마친 교차 인코더를 허깅페이스 허브에 업로드**

In [ ]:
from huggingface_hub import HfApi

api = HfApi()
repo_id = "klue-roberta-small-cross-encoder"
api.create_repo(repo_id=repo_id)

api.upload_folder(
    folder_path=model_save_path,
    repo_id=f"본인의 허깅페이스 아이디 입력/{repo_id}",
    repo_type="model",
)